## 1. Markdown 가져오기

- "unstructured[md]" nltk 패키지를 설치함으로써 `langchain_community.document_loaders.UnstructuredMarkdownLoader` 사용

In [1]:
from pathlib import Path

from langchain_community.document_loaders import UnstructuredMarkdownLoader


DATA_DIR = Path("../data")
tax_law_md_file_path = DATA_DIR / "income-tax-law-parsed-by-upstage.md"
loader = UnstructuredMarkdownLoader(tax_law_md_file_path)

In [2]:
docs = loader.load()
print("불러온 문서 개수:", len(docs))

불러온 문서 개수: 1


## 2. Markdown 문서 나누기
- `MarkdownTextSplitter` 를 사용해서 문서를 잘 나누도록 하기

In [3]:
from langchain_text_splitters import MarkdownTextSplitter


text_splitter = MarkdownTextSplitter(
    chunk_size=1500,  # 한 청크의 최대 길이(문자 기준)
    chunk_overlap=250,  # 청크 간 겹침(문자 기준)
    add_start_index=True,  # 원문 내 시작 인덱스 메타데이터 추가
)
split_docs = loader.load_and_split(text_splitter)
print(len(split_docs))

297


## 3. VectorDB 에 나눈 문서 저장하기

### 3.1. Embeddings 로 bge-m3 모델을 사용한다.

BGE-M3는 XLM-RoBERTa 아키텍처를 기반으로 하며, 다기능성(Multi-Functionality), 다언어성(Multi-Linguality), 다중 세분성(Multi-Granularity) 측면에서 뛰어난 다재다능함으로 구별된다.

- 다기능성(Multi-Functionality): 임베딩 모델의 세 가지 일반적인 검색 기능인 밀집 검색(dense retrieval), 다중 벡터 검색(multi-vector retrieval), 희소 검색(sparse retrieval)을 동시에 수행할 수 있다.
    - 이것 때문에 VectorDB 를 만들 때 Embeddings 로 사용하면 좋다.
- 다언어성(Multi-Linguality): 100개 이상의 언어를 지원할 수 있다.
- 다중 세분성(Multi-Granularity): 짧은 문장부터 최대 8192개 토큰의 긴 문서까지 다양한 세분성의 입력을 처리할 수 있다.

In [4]:
from langchain_ollama import OllamaEmbeddings


embedding_function = OllamaEmbeddings(model="bge-m3:567m")

### 3.2. chroma 를 이용해서 VectorDB 구축

In [5]:
from langchain_chroma import Chroma


chroma_persistent_dir = DATA_DIR / "chroma_db"
vector_store = Chroma(
    embedding_function=embedding_function,
    persist_directory=chroma_persistent_dir,
    collection_name="korean_income_tax_law",
)

In [16]:
# bge-m3 를 사용했을 때, 총 297개의 문서를 저장하는데 약 32초가 걸렸다.
# vector_store.add_documents(split_docs)
# 처음 생성 후에는 다시 생성할 필요없으므로 주석 처리

In [7]:
retriever = vector_store.as_retriever(kwargs={"k": 3})

## 4. Simple RAG LangGraph 만들기

- `retrieve` 노드와 `generate` 노드가 있어야 한다.
    - `retrieve`: vector db 에서, 사용자의 질문과 관련된 정보를 가져오는 일을 하는 Node
    - `generate`: 사용자의 질문과 검색해온 정보를 가지고 답변을 생성하는 Node

### 4.1 Graph 의 State 만들기

In [8]:
from typing import TypedDict

from langchain_core.documents import Document


class AgentState(TypedDict):
    query: str  # 사용자의 질문
    context: list[Document]  # 질문으로 검색한 문서
    answer: str  # 문서 + 질문에 대한 llm 의 답변

### 4.2. State 로 GraphBuilder 만들기

In [9]:
from langgraph.graph import StateGraph


graph_builder = StateGraph(AgentState)

### 4.3. retrieve Node 만들기

In [10]:
def retrieve(state: AgentState) -> AgentState:
    query = state["query"]
    retrieved_docs = retriever.invoke(query)
    return {"context": retrieved_docs}


graph_builder.add_node("retrieve", retrieve)

### 4.4. generate Node 만들기

In [11]:
from langchain_ollama import ChatOllama


llm = ChatOllama(model="gpt-oss:20b")

In [12]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate


system_prompt = """역할: 당신은 한국의 소득세법 전문가입니다.
---
지시: 다음 정보를 바탕으로 사용자의 질문에 답하세요.
---
정보: {context}
---
사용자 질문: {query}
"""


def generate(state: AgentState) -> AgentState:
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt),
        ]
    )
    rag_chain = prompt | llm | StrOutputParser()
    answer = rag_chain.invoke(
        {
            "query": state["query"],
            "context": state["context"],
        }
    )
    return {"answer": answer}


graph_builder.add_node("generate", generate)

### 4.5. Edge 로 연결해서 그래프 완성하기

In [ ]:
from langgraph.graph import END, START


graph_builder.add_edge(START, "retrieve")
graph_builder.add_edge("retrieve", "generate")
graph_builder.add_edge("generate", END)
graph = graph_builder.compile()

# 이전에 graph 를 그리는 방법
# from IPython.display import Image, display
#
# display(Image(graph.get_graph().draw_mermaid_png()))

graph

In [14]:
initial_state = {"query": "연봉 5천만원인 직장인의 소득세는 얼마입니까?"}
graph.invoke(initial_state)

{'query': '연봉 5천만원인 직장인의 소득세는 얼마입니까?',
 'context': [Document(id='22ed77a4-2054-46d1-a6d6-fcc2a995584f', metadata={'source': '../data/income-tax-law-parsed-by-upstage.md', 'start_index': 21510}, page_content='과세기간의 총수입금액에서 이에 사용된 필요경비를 공제한 금액으로 하며, 필요경비가 총 수입금액을 초과하는 경우 그 초과하는 금액을 "결손금"이라 한다. ③ 제1항 각 호에 따른 사업의 범위에 관하여는 이 법에 특별한 규정이 있는 경우 외에는 「통계법」 제22조에 따라 통계청장이 고시하는 한국표준산업분류에 따르고, 그 밖의 사업소득의 범위에 관하여 필요한 사항은 대통령령으로 정한다.# [전문개정 2009. 12. 31.]제20조(근로소득) ① 근로소득은 해당 과세기간에 발생한 다음 각 호의 소득으로 한다. <개정 2016. 12. 20., 2024. 12. 31.>- 1. 근로를 제공함으로써 받는 봉급 · 급료 · 보수 · 세비 · 임금 · 상여 · 수당과 이와 유사한 성질의 급여 - 2. 법인의 주주총회 · 사원총회 또는 이에 준하는 의결기관의 결의에 따라 상여로 받는 소득 - 3. 「법인세법」에 따라 상여로 처분된 금액 - 4. 퇴직함으로써 받는 소득으로서 퇴직소득에 속하지 아니하는 소득 - 5. 종업원등 또는 대학의 교직원이 지급받는 직무발명보상금(제21조제1항제22호의2에 따른 직무발명보상금은 제 - 외한다) - 6. 사업자나 법인이 생산 · 공급하는 재화 또는 용역을 그 사업자나 법인(「독점규제 및 공정거래에 관한 법률」 에 따 - 른 계열회사를 포함한다)의 사업장에 종사하는 임원등에게 대통령령으로 정하는 바에 따라 시가보다 낮은 가격 - 으로 제공하거나 구입할 수 있도록 지원함으로써 해당 임원등이 얻는 이익 - ② 근로소득금액은 제1항 각 호의 소득의 금액의 합계액(비과세소득의 금액은 

### 4.6. `add_sequence` 를 사용해보기

In [ ]:
# add_sequence 를 사용해서 아래와 같이 표현할 수도 있음
seq_graph_builder = StateGraph(AgentState).add_sequence([retrieve, generate])
seq_graph_builder.add_edge(START, "retrieve")
seq_graph_builder.add_edge("generate", END)
seq_graph_builder.compile()

###